# TODO
- Parallelize cluster_approach over states
- Parallelize grid_approach over states
- Make two different .py files: cluster approach, grid approach.
- Make into folder: generating 'bounding_boxes'

- Make a separate prep_herb.py

- Correct eps size into haversine distance calculations

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [2]:
# Get Herbarium data. Find some candidate locations.
occ = pd.read_csv("/data/cher/universe7/herbarium/data/MO-herbarium/occurrence.txt", sep="\t", on_bad_lines='skip')
occ = occ[[
    'gbifID', 'occurrenceID', # Joining Keys
    'speciesKey', 'species', # species
    'year', 'month', 'day', # date
    'habitat', 'locality', # descriptive text. I think this is most of the route / habitat info we would want. Looks like already OCR processed?
    'countryCode', 'stateProvince', 'county', 'municipality', # administrative
    'georeferenceSources','hasCoordinate', 'hasGeospatialIssues' ,'decimalLatitude', 'decimalLongitude']] # geospatial
    # 'level0Gid', 'level0Name', 'level1Gid', 'level1Name', 'level2Gid', 'level2Name', 'level3Gid', 'level3Name',]] # Has NAs. Above is more reliable.

habitat_info = occ[~occ['habitat'].isna()].copy()

/tmp/ipykernel_329770/903731494.py:2: DtypeWarning: Columns (49,56,57,59,60,61,63,64,65,66,67,68,73,74,77,78,79,83,89,91,92,95,97,98,99,100,101,102,103,105,106,107,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,135,137,138,140,141,142,143,144,145,146,147,149,150,151,152,153,154,155,160,162,163,164,167,168,171,173,174,177,178,182,183,184,185,186,189,190,191,192,193,194,195,196,197,198,199,200,204,208,209,210,211,220,221) have mixed types. Specify dtype option on import or set low_memory=False.
  occ = pd.read_csv("/data/cher/universe7/herbarium/data/MO-herbarium/occurrence.txt", sep="\t", on_bad_lines='skip')


In [4]:
# Random location
random_row = habitat_info[~habitat_info['habitat'].isna()].iloc[-4]
lat, lon = random_row['decimalLatitude'], random_row['decimalLongitude']

NAIP 
- Only 3 band RGB through WMS

In [13]:
import urllib.request
import rasterio
import matplotlib.pyplot as plt
import numpy as np


image_url = f"https://basemap.nationalmap.gov/arcgis/services/USGSImageryOnly/MapServer/WMSServer?service=WMS&version=1.1.1&request=GetMap&layers=0&styles=&width=256&height=256&srs=EPSG:4326&bbox={lon-0.005},{lat-0.005},{lon+0.005},{lat+0.005}&format=image/png"
out_file = f'/data/cher/universe7/herbarium/data/naip/1.png'

urllib.request.urlretrieve(image_url, out_file)

with rasterio.open(out_file) as src:
    # Get the number of bands
    num_bands = src.count
    print(f'The TIFF file has {num_bands} bands.')

('/data/cher/universe7/herbarium/data/naip/1.tiff',
 <http.client.HTTPMessage at 0x7f105f5980e0>)

Sentinel2
- Create csv file to push into CVGlobal 
    1) Cluster points state-by-state using DBSCAN into max areas of 512 x 512 meters --> cell
    2) Find center point of each cell

#### USA

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
from sklearn.cluster import DBSCAN
import numpy as np
from tqdm import tqdm
import os

1) DBSCAN still generates clustered grid cells outside of 512 meters.
- Instead we will do the following --> 
    1) generate DBSCAN clusters. Find points within those centers. 
    2) Points outside centers --> DBSCAN. Find points within those centers.

2) Fast --> Create grid for state. Find overlap of points. Return just those grids

In [134]:
# Get general info.
num_bbox_per_state = clusters['state_cluster'].str.split('_').str[0].value_counts()
num_images_per_state = cluster_assignments['state_cluster'].str.split('_').str[0].value_counts()
num_images_per_cluster = num_images_per_state / num_bbox_per_state

print(num_bbox_per_state, num_images_per_state, num_images_per_cluster)

display(num_images_per_cluster.sort_values(ascending = False))

state_cluster
Michigan    788
Name: count, dtype: int64 state_cluster
Michigan    4479
Name: count, dtype: int64 state_cluster
Michigan    5.68401
Name: count, dtype: float64


state_cluster
Michigan    5.68401
Name: count, dtype: float64

In [1]:
import pandas as pd
import geopandas as gpd
from scipy.spatial.distance import pdist, squareform
from shapely.geometry import Point

In [4]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lon1, lat1, lon2, lat2):
    # Convert latitude and longitude from degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Radius of Earth in kilometers. Use 3956 for miles. Determines return value units.
    r = 6371
    distance = r * c
    return distance

# Coordinates for the two points
lon1, lat1 = -90.67098999999999, 38.45601
lon2, lat2 = -90.66098999999998, 38.45601

# Calculate distance
distance_km = haversine(lon1, lat1, lon2, lat2)
print(f"Distance: {distance_km:.2f} km")

Distance: 0.87 km


In [2]:
grid = gpd.read_file("/data/cher/universe7/herbarium/data/geocell/grid.geojson")
grid_key = pd.read_csv("/data/cher/universe7/herbarium/data/geocell/grid_key.csv")

In [14]:
grid_w_observations = grid_within_state[grid_within_state['key'].isin(observations_in_cells['key'])]
grid['lon'], grid['lat'] = grid.centroid.x, grid.centroid.y
output_bbox_pt[['key', 'lon', 'lat']]

/tmp/ipykernel_732265/2479746997.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid['lon'], grid['lat'] = grid.centroid.x, grid.centroid.y


,key,geometry,lon,lat
0,Virginia_0,"POLYGON ((-80.32119 36.54086, -80.32119 36.545...",-80.324115,36.543162
1,Virginia_1,"POLYGON ((-80.31533 36.54086, -80.31533 36.545...",-80.318262,36.543162
2,Virginia_2,"POLYGON ((-80.30948 36.54086, -80.30948 36.545...",-80.312408,36.543162
3,Virginia_3,"POLYGON ((-80.30363 36.54086, -80.30363 36.545...",-80.306554,36.543162
4,Virginia_4,"POLYGON ((-80.29777 36.54086, -80.29777 36.545...",-80.300700,36.543162
...,...,...,...,...
420715,Virginia_420715,"POLYGON ((-78.32506 39.45603, -78.32506 39.460...",-78.327982,39.458333
420716,Virginia_420716,"POLYGON ((-78.34847 39.46064, -78.34847 39.465...",-78.351397,39.462946
420717,Virginia_420717,"POLYGON ((-78.34262 39.46064, -78.34262 39.465...",-78.345543,39.462946
420718,Virginia_420718,"POLYGON ((-78.33676 39.46064, -78.33676 39.465...",-78.339690,39.462946


In [15]:
grid_key

,604cee1e-de47-4bc1-995a-16a184ae23d0,Virginia_363421
0,61d5d9b3-f30c-4ccc-861d-572d2d4cf594,Virginia_162625
1,62eaea3a-ed7e-4780-8c65-f8449218573b,Virginia_78531
2,6370692e-95d7-450d-b794-0bff0411abdb,Virginia_42488
3,645d9394-6b6c-41ff-a3c9-7cac6cea561b,Virginia_394992
4,67ffb10c-a349-4425-a093-e51c21cc4ad1,Virginia_278232
...,...,...
10992,917e250c-5bc2-4061-92ed-e88fb7911ee0,Virginia_85090
10993,6b14fda1-eb5d-43dd-ac55-6aad2c9344c1,Virginia_278232
10994,933ef55a-027f-40af-bbc9-bd513d7700d6,Virginia_359956
10995,6d6ae8e9-9405-41d7-ab77-08daab6faa22,Virginia_374426


In [71]:
### sanity checks
# Are places assigned to the same cluster within 512 meters of one another?
habitat_gdf_w_bbox_cluster = gdf.merge(cluster_assignments, on = ['occurrenceID'])

ex = habitat_gdf_w_bbox_cluster[habitat_gdf_w_bbox_cluster['stateProvince'] == 'New York']

In [ ]:
# Use grid cells across US that have occurrences for query of images.
# sentinal2_key.csv -- Source - gbif, key - numeric_key, lat, lon  
# obs_id2key .csv -- [Source, obs_id, key, bbox_bounds]